In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from enum import Enum
import math

class FlightMode(Enum):
    fmIdle          = 0
    fmLaunched      = 1
    fmDescending    = 2
    fmLanded        = 3

class Delta():
    def __init__(self, latitude, longitude):
        self.latitude = latitude 
        self.longitude = longitude
        
class Predictor(object):
    def __init__(self, LandingAltitude, DefaultCDA):
        self.SlotSize = 100
        self.SlotCount = 60000 // self.SlotSize
        self.Deltas = []
        self.FlightMode = FlightMode.fmIdle
        self.PreviousPosition = {'time': '00:00:00', 'lat': 0.0, 'lon': 0.0, 'alt': 0, 'sats': 0, 'fixtype': 0}
        self.MinimumAltitude = 0
        self.MaximumAltitude = 100000000000
        self.AscentRate = 0
        self.LandingAltitude = LandingAltitude
        self.LandingLatitude = 0.0
        self.LandingLongitude = 0.0
        self.PollPeriod = 5
        self.Counter = 0
        self.CDA = DefaultCDA
        for i in range(self.SlotCount):
            self.Deltas.append(Delta(0,0))


    def GetSlot(self, Altitude):
        Slot = int(Altitude // self.SlotSize)

        if Slot < 0:
            Slot = 0
        if Slot >= self.SlotSize:
            Slot = self.SlotSize-1

        return Slot

    def CalculateAirDensity(self, Altitude):
        if Altitude < 11000.0:
            # below 11Km - Troposphere
            Temperature = 15.04 - (0.00649 * Altitude)
            Pressure = 101.29 * pow((Temperature + 273.1) / 288.08, 5.256)
        elif Altitude < 25000.0:
            # between 11Km and 25Km - lower Stratosphere
            Temperature = -56.46
            Pressure = 22.65 * math.exp(1.73 - ( 0.000157 * Altitude))
        else:
            # above 25Km - upper Stratosphere
            Temperature = -131.21 + (0.00299 * Altitude)
            Pressure = 2.488 * math.pow((Temperature + 273.1) / 216.6, -11.388)

        return Pressure / (0.2869 * (Temperature + 273.1))

    def CalculateDescentRate(self, Weight, CDTimesArea, Altitude):
        Density = self.CalculateAirDensity(Altitude)
	
        return math.sqrt((Weight * 9.81)/(0.5 * Density * CDTimesArea))
            
    def CalculateCDA(self, Weight, Altitude, DescentRate):
        if DescentRate > 0.0:
            Density = self.CalculateAirDensity(Altitude)
	
            # printf("Alt %.0lf, Rate %.1lf, CDA %.1lf\n", Altitude, DescentRate, (Weight * 9.81)/(0.5 * Density * DescentRate * DescentRate));
        
            return (Weight * 9.81)/(0.5 * Density * DescentRate * DescentRate)
        else:
            return self.CDA
            # (lat, long, alt)
            
    def CalculateLandingPosition(self, Latitude, Longitude, Altitude):
        TimeTillLanding = 0;
	
        Slot = self.GetSlot(Altitude);
        DistanceInSlot = Altitude + 1 - Slot * self.SlotSize
        while Altitude > self.LandingAltitude:
            Slot = self.GetSlot(Altitude)
            if Slot == self.GetSlot(self.LandingAltitude):
                DistanceInSlot = Altitude - self.LandingAltitude
		
            DescentRate = self.CalculateDescentRate(1.0, self.CDA, Altitude)
            
            TimeInSlot = DistanceInSlot / DescentRate
            
            Latitude += self.Deltas[Slot].latitude * TimeInSlot
            Longitude += self.Deltas[Slot].longitude * TimeInSlot
            # printf("SLOT %d: alt %lu, lat=%lf, long=%lf, rate=%lf, dist=%lu, time=%lf\n", Slot, Altitude, Latitude, Longitude, DescentRate, DistanceInSlot, TimeInSlot);
            
            TimeTillLanding = TimeTillLanding + TimeInSlot
            Altitude -= DistanceInSlot
            DistanceInSlot = self.SlotSize
        
        return {'pred_lat': Latitude, 'pred_lon': Longitude ,'TTL': TimeTillLanding}

    def AddGPSPosition(self, Position):
        Result = None
        
        if Position['sats'] >= 4:
            self.Counter = self.Counter + 1
            if self.Counter >= self.PollPeriod:
                self.Counter = 0
                
                if Position['alt'] <= 0:
                    self.AscentRate = 0
                else:
                    self.AscentRate = self.AscentRate * 0.7 + (Position['alt'] - self.PreviousPosition['alt']) * 0.3;

                if (Position['alt'] < self.MinimumAltitude) or (self.MinimumAltitude == 0):
                    self.MinimumAltitude = Position['alt']
                    
                if Position['alt'] > self.MaximumAltitude:
                    self.MaximumAltitude = Position['alt']               

                if (self.AscentRate >= 1.0) and (Position['alt'] > (self.MinimumAltitude+150)) and (self.FlightMode == FlightMode.fmIdle):
                    self.FlightMode = FlightMode.fmLaunched
                    print("*** LAUNCHED ***");
            
                if (self.AscentRate < -10.0) and (self.MaximumAltitude >= (self.MinimumAltitude+2000)) and (self.FlightMode == FlightMode.fmLaunched):
                    self.FlightMode = FlightMode.fmDescending
                    print("*** DESCENDING ***");

                if (self.AscentRate >= -0.1) and (Position['alt'] <= self.LandingAltitude+2000) and (self.FlightMode == FlightMode.fmDescending):
                    self.FlightMode = FlightMode.fmLanded
                    print("*** LANDED ***")
                   
                if self.FlightMode == FlightMode.fmLaunched:
                    # Going up - store deltas
                    
                    Slot = self.GetSlot(Position['alt']/2 + self.PreviousPosition['alt']/2);
                        
                    # Deltas are scaled to be horizontal distance per second (i.e. speed)
                    self.Deltas[Slot].latitude = (Position['lat'] - self.PreviousPosition['lat']) / self.PollPeriod
                    self.Deltas[Slot].longitude = (Position['lon'] - self.PreviousPosition['lon']) / self.PollPeriod
                    
                    print("Slot " + str(Slot) + " = " + str(Position['alt']) + "," + str(self.Deltas[Slot].latitude) + "," + str(self.Deltas[Slot].longitude))
                elif self.FlightMode == FlightMode.fmDescending:
                    # Coming down - try and calculate how well chute is doing

                    self.CDA = (self.CDA * 4 + self.CalculateCDA(1.0, Position['alt']/2 + self.PreviousPosition['alt']/2, (self.PreviousPosition['alt'] - Position['alt']) / self.PollPeriod)) / 5
                
                
                if (self.FlightMode == FlightMode.fmLaunched) or (self.FlightMode == FlightMode.fmDescending):
                    Result = self.CalculateLandingPosition(Position['lat'], Position['lon'], Position['alt']);
                    print(Result)

                    # GPS->PredictedLandingSpeed = CalculateDescentRate(Config.payload_weight, GPS->CDA, Config.LandingAltitude);
				
                    # printf("Expected Descent Rate = %4.1lf (now) %3.1lf (landing), time till landing %d\n", 
                            # CalculateDescentRate(Config.payload_weight, GPS->CDA, GPS->Altitude),
                            # GPS->PredictedLandingSpeed,
                            # GPS->TimeTillLanding);

                    # printf("Current    %f, %f, alt %" PRId32 "\n", GPS->Latitude, GPS->Longitude, GPS->Altitude);
                    # printf("Prediction %f, %f, CDA %lf\n", GPS->PredictedLatitude, GPS->PredictedLongitude, GPS->CDA);


                print('PREDICTOR: ' + str(Position['time']) + ', ' + "{:.5f}".format(Position['lat']) + ', ' + "{:.5f}".format(Position['lon']) + ', ' + str(Position['alt']) + ', ' + str(Position['sats']))

                self.PreviousPosition = Position.copy()
                
        return Result

#1. Start at point A
#2. Move to point B
#3. At point B, CalculateLandingPosition

############# START OF PREDICTION

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from collections import defaultdict
from shapely.geometry import Point, Polygon

region_0 = pd.read_csv("/content/drive/Shareddrives/Bruin Space/Projects/Overseer/2020-2021/Spring/Software/NewCoords/Square 0.csv")["Polygon/outerBoundaryIs/LinearRing/coordinates"]
region_1 = pd.read_csv("/content/drive/Shareddrives/Bruin Space/Projects/Overseer/2020-2021/Spring/Software/NewCoords/Square 1.csv")["Polygon/outerBoundaryIs/LinearRing/coordinates"]
region_3 = pd.read_csv("/content/drive/Shareddrives/Bruin Space/Projects/Overseer/2020-2021/Spring/Software/NewCoords/Square 3.csv")["Polygon/outerBoundaryIs/LinearRing/coordinates"]
region_4 = pd.read_csv("/content/drive/Shareddrives/Bruin Space/Projects/Overseer/2020-2021/Spring/Software/NewCoords/Square 4.csv")["Polygon/outerBoundaryIs/LinearRing/coordinates"]
region_5 = pd.read_csv("/content/drive/Shareddrives/Bruin Space/Projects/Overseer/2020-2021/Spring/Software/NewCoords/Square 5.csv")["Polygon/outerBoundaryIs/LinearRing/coordinates"]
region_8 = pd.read_csv("/content/drive/Shareddrives/Bruin Space/Projects/Overseer/2020-2021/Spring/Software/NewCoords/Square 8.csv")["Polygon/outerBoundaryIs/LinearRing/coordinates"]
region_9 = pd.read_csv("/content/drive/Shareddrives/Bruin Space/Projects/Overseer/2020-2021/Spring/Software/NewCoords/Square 9.csv")["Polygon/outerBoundaryIs/LinearRing/coordinates"]
all_regions = [region_0, region_1, region_3, region_4, region_5, region_8, region_9]

In [4]:
#Processing coordinates, constructing shapes
mapped_regions = defaultdict(list)
#{key: values, key: values}
count = 0
for region in all_regions: #region_0
    for coord_set in region: #Iterate through every shape in the region
        split_coords = coord_set.split()
        shape = []
        for s in split_coords:
            lat_long_0 = s.split(",") # abc,def,g --> [abc, def, g]
            shape.append(Point(float(lat_long_0[1]), float(lat_long_0[0])))
        curr_poly = Polygon(shape)
        mapped_regions[count] = curr_poly
        count += 1

In [ ]:
class SimulateMovement():
  def __init__(self, lat, lon, alt):
    self.lat = lat
    self.lon = lon
    self.alt = alt

  def changePosition(self):
    self.lon -= 0.005
  
  def increaseAltitude(self):
    self.alt += 100

  def decreaseAltitude(self):
    self.alt -= 1000

In [ ]:
def run_tests(x, y):
    pred = Predictor(40000, 17.5)
    sim = SimulateMovement(33.97366, -116.1925, 30000)
    

    def check_within_red(curr_loc): #Check if the balloon is inside a red zone based upon above regions
        for shape in mapped_regions:
            if curr_loc.within(mapped_regions[shape]):
                return True

    def update_loc(c):
        return Point(pred.LandingLatitude, pred.LandingLongitude)

    stop = 0
    while True:
        while sim.alt < pred.MaximumAltitude: #not yet at max altitude
            print("NOT AT MAX ALT")
            sim.changePosition()
            sim.changeAltitude()
            d = {'time': 0, 'lat': sim.lat, 'lon': sim.lon, 'alt': sim.alt, 'sats': 5, 'fixtype': 0} 
            print(predictor.AddGPSPosition(d))
            continue
        while check_within_red(update_loc(pred)):
            sim.changePosition()
            sim.changeAltitude()
            print("Already in a red zone")
            #at max altitude, but already in a red zone
            continue 
        while True: # the balloon is in a white zone and also above max altitude
            print("in a white zone")
            if check_within_red(update_loc(pred)):
                sim.changePosition()
                sim.changeAltitude()
                pred.cutdown()
                stop = 1
                break
        if stop:
            break
run_tests(35.3084, -118.57268)
predictor = Predictor(40000, 17.5)
# CalculateLandingPosition(Latitude, Longitude, Altitude)
lat = 35.0 #current latitude
longi = -117.15
alt = 60000
time = 0
for i in range(1, 30):
  d = {'time': str(time) + str(i), 'lat': lat, 'lon': longi, 'alt': alt, 'sats': 5, 'fixtype': 0}
  longi -= 0.1
  alt += 100 #increasing altitude means ascending
  print(predictor.AddGPSPosition(d))


In [ ]:
pred = Predictor(0, 17.5) #Initialize the predictor
sim = SimulateMovement(33.97366, -116.1925, 0) #Initialize new balloon WITH Starting longitude and latitude, and altitude

def check_within_red(curr_loc): #Check if the balloon is inside a red zone based upon above regions
    for shape in mapped_regions:
        if curr_loc.within(mapped_regions[shape]):
            return True

def update_loc(c):
    return Point(c.LandingLatitude, c.LandingLongitude)


#CODE BEGINS
while True:
  while sim.alt < pred.MaximumAltitude: #not yet at max altitude
  #sim.alt = balloon's altitude
  #pred.MaximumAltitude = the maximum altitude that it can go
      print("NOT AT MAX ALT")

      #simulate balloon moving
      sim.changePosition()
      sim.increaseAltitude()
      
      #Complicated code needed for this to work -- JSON format for inputting current location
      d = {'time': 0, 'lat': sim.lat, 'lon': sim.lon, 'alt': sim.alt, 'sats': 5, 'fixtype': 0} 

      # ex. The balloon was at (33.97366, -116.1925, 0), now it is at (33.9731, -116.1925, 1000)
      res = pred.AddGPSPosition(d)

      if res:
        print("CURR POSITION: " + str(sim.lat) + ", " + str(sim.lon))
        print("PREDICTED: " + str(res['pred_lat']) + ", " + str(res['pred_lon']))
      if res and check_within_red(Point(res['pred_lat'], res['pred_lon'])):
        print("Already in a red zone")
        break
      else:
        print("in a white zone")
      continue
  print("FINISH") # Write to GPIO
  break


  ||||||||||P||||  P     X  X  X  X  X

NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
PREDICTOR: 0, 33.97366, -116.21750, 500, 5
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
*** LAUNCHED ***
Slot 7 = 1000,0.0,-0.0049999999999954525
{'pred_lat': 33.97366, 'pred_lon': -116.74566450051579, 'TTL': 1018.6561138883768}
PREDICTOR: 0, 33.97366, -116.24250, 1000, 5
CURR POSITION: 33.97366, -116.24249999999995
PREDICTED: 33.97366, -116.74566450051579
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
in a white zone
NOT AT MAX ALT
Slot 12 = 1500,0.0,-0.0049999999999954525
{'pred_lat': 33.97366, 'pred_lon': -117.26162891403642, 'TTL': 1509.6025994393922}
PREDICTOR: 0, 33.97366, -116.26750, 1500, 5
CURR POSITION: 33.97366, -116.26749999999993
PREDICTED: 33.97366, -1

In [ ]:

predictor = Predictor(40000, 17.5)
# CalculateLandingPosition(Latitude, Longitude, Altitude)
lat = 35.0 #current latitude
longi = -117.15
alt = 60000
time = 0
for i in range(1, 30):
  d = {'time': str(time) + str(i), 'lat': lat, 'lon': longi, 'alt': alt, 'sats': 5, 'fixtype': 0}
  longi -= 0.1
  alt += 100 #increasing altitude means ascending
  print(predictor.AddGPSPosition(d))

None
None
None
None
PREDICTOR: 05, 35.00000, -117.55000, 60400, 5
None
None
None
None
None
*** LAUNCHED ***
Slot 99 = 60900,0.0,-0.09999999999999432
{'pred_lat': 35.0, 'pred_lon': -150.66796854999893, 'TTL': 326.17968550000836}
PREDICTOR: 010, 35.00000, -118.05000, 60900, 5
{'pred_lat': 35.0, 'pred_lon': -150.66796854999893, 'TTL': 326.17968550000836}
None
None
None
None
Slot 99 = 61400,0.0,-0.09999999999999432
{'pred_lat': 35.0, 'pred_lon': -151.0093588444744, 'TTL': 324.59358844476327}
PREDICTOR: 015, 35.00000, -118.55000, 61400, 5
{'pred_lat': 35.0, 'pred_lon': -151.0093588444744, 'TTL': 324.59358844476327}
None
None
None
None
Slot 99 = 61900,0.0,-0.09999999999999432
{'pred_lat': 35.0, 'pred_lon': -151.33810626972507, 'TTL': 322.8810626972703}
PREDICTOR: 020, 35.00000, -119.05000, 61900, 5
{'pred_lat': 35.0, 'pred_lon': -151.33810626972507, 'TTL': 322.8810626972703}
None
None
None
None
Slot 99 = 62400,0.0,-0.09999999999999432
{'pred_lat': 35.0, 'pred_lon': -151.6551726690514, 'TTL':